# 01 U-Net基线模型实验

## 目标
- 实现U-Net基线模型（仅L1损失，无GAN）
- 数据增强消融实验（无增强、基础增强、强增强）
- 训练过程可视化（损失曲线、三联图）
- 评估指标记录（PSNR、SSIM、MAE）

## 模型架构
- 与Pix2Pix生成器相同的U-Net结构
- 编码器：C64-C128-C256-C512-C512-C512-C512-C512
- 解码器：CD512-CD512-CD512-C512-C256-C128-C64
- Skip Connections：编码器第i层连接到解码器第n-i层

## 训练设置
- 优化器：Adam, lr=2e-4
- 学习率调度：线性衰减（从epoch 100开始）
- 训练轮数：200 epochs
- Batch size：1
- 损失函数：仅L1损失



## Colab环境设置

### 📦 数据同步方案（三选一）

#### 方案一：Google Drive for Desktop（推荐，最简单）
1. **安装 Google Drive for Desktop**：https://www.google.com/drive/download/
2. **设置同步文件夹**：
   - 将项目文件夹添加到Google Drive同步目录
   - 本地修改会自动同步到云端
   - 在Colab中直接使用同步后的文件，无需手动上传

#### 方案二：Git + GitHub（推荐，适合代码）
1. **将代码推送到GitHub**（数据文件用Git LFS或单独上传）
2. **在Colab中直接克隆**：代码会自动从GitHub拉取最新版本
3. **优点**：版本控制、自动同步、无需压缩上传

#### 方案三：手动上传（备选）
如果使用手动上传，请将以下文件压缩：
- `src/` 目录（源代码）
- `data/splits/cityscapes_split_seed42.json`（数据划分文件）
- `data/processed/` 目录（图像数据，可压缩为 `data_processed.zip`）

---

### 📝 使用说明

**如果使用方案一（Google Drive同步）**：
- 确保项目文件夹已在Google Drive中同步
- 直接运行下面的代码，会自动检测并使用同步的文件

**如果使用方案二（GitHub）**：
- 在下面的代码中设置 `USE_GITHUB = True` 并填写仓库地址
- 代码会自动克隆最新版本

**如果使用方案三（手动上传）**：
- 上传压缩包到Google Drive
- 代码会自动解压



In [1]:
# ============================================
# Colab环境设置：支持多种同步方案
# ============================================

# 先导入依赖
import os
from pathlib import Path
import zipfile
import shutil
import subprocess
import sys
from google.colab import drive

# ========== 配置区域 ==========
# 选择同步方案：'github', 'drive_sync', 'manual'
SYNC_METHOD = 'github'  # 或 'drive_sync' 或 'manual'

# GitHub配置（如果使用方案二）
GITHUB_REPO = "AltheD/Image-to-Image-Translation-Experiment"  # 替换为你的GitHub仓库地址
GITHUB_BRANCH = "main"  # 或 "master"

# Google Drive配置（如果使用方案一或三）
DRIVE_PROJECT_DIR = Path("/content/drive/MyDrive/Image-to-Image-Translation-Experiment")
# ==============================

COLAB_WORK_DIR = Path("/content/Image-to-Image-Translation-Experiment")

# 1. 挂载Google Drive（所有方案都需要）
print("正在挂载Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive挂载完成！")

# 2. 根据选择的方案设置项目
if SYNC_METHOD == 'github':
    # 方案二：从GitHub克隆
    print(f"\n📦 从GitHub克隆项目: {GITHUB_REPO}")
    if COLAB_WORK_DIR.exists():
        print("⚠️  项目目录已存在，拉取最新代码")
        subprocess.run(["git", "-C", str(COLAB_WORK_DIR), "pull"], check=True)
    else:
        repo_url = f"https://github.com/{GITHUB_REPO}.git"
        subprocess.run(["git", "clone", "-b", GITHUB_BRANCH, repo_url, str(COLAB_WORK_DIR)], check=True)
        print("✅ GitHub克隆完成！")

    # 检查数据文件（GitHub通常不包含大数据文件）
    if not (COLAB_WORK_DIR / "data" / "processed").exists():
        print("\n⚠️  检测到数据文件缺失，尝试从Google Drive复制...")
        if (DRIVE_PROJECT_DIR / "data" / "processed").exists():
            (COLAB_WORK_DIR / "data").mkdir(parents=True, exist_ok=True)
            if (COLAB_WORK_DIR / "data" / "processed").exists():
                if (COLAB_WORK_DIR / "data" / "processed").is_symlink():
                    print("✅ 数据目录符号链接已存在")
                else:
                    shutil.rmtree(COLAB_WORK_DIR / "data" / "processed")
                    os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                              COLAB_WORK_DIR / "data" / "processed")
                    print("✅ 数据目录符号链接创建完成！")
            else:
                os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                          COLAB_WORK_DIR / "data" / "processed")
                print("✅ 数据目录符号链接创建完成！")
        else:
            print("❌ 未找到数据文件，请确保已上传到Google Drive")

    os.chdir(COLAB_WORK_DIR)

elif SYNC_METHOD == 'drive_sync':
    # 方案一：使用Google Drive同步的文件（直接使用，无需解压）
    print("\n📁 使用Google Drive同步的文件...")
    if not DRIVE_PROJECT_DIR.exists():
        print(f"❌ 未找到项目目录: {DRIVE_PROJECT_DIR}")
        print("   请确保已安装Google Drive for Desktop并同步项目文件夹")
    else:
        print(f"✅ 找到项目目录: {DRIVE_PROJECT_DIR}")
        # 创建符号链接或直接使用（如果数据在Drive中）
        if not COLAB_WORK_DIR.exists():
            COLAB_WORK_DIR.mkdir(parents=True, exist_ok=True)

        # 复制源代码（小文件）
        if (DRIVE_PROJECT_DIR / "src").exists():
            if (COLAB_WORK_DIR / "src").exists():
                shutil.rmtree(COLAB_WORK_DIR / "src")
            shutil.copytree(DRIVE_PROJECT_DIR / "src", COLAB_WORK_DIR / "src")
            print("✅ 源代码已复制")

        # 数据文件使用符号链接（节省空间和时间）
        (COLAB_WORK_DIR / "data").mkdir(parents=True, exist_ok=True)
        if (DRIVE_PROJECT_DIR / "data" / "splits").exists():
            (COLAB_WORK_DIR / "data" / "splits").mkdir(parents=True, exist_ok=True)
            if not (COLAB_WORK_DIR / "data" / "splits" / "cityscapes_split_seed42.json").exists():
                shutil.copy2(DRIVE_PROJECT_DIR / "data" / "splits" / "cityscapes_split_seed42.json",
                            COLAB_WORK_DIR / "data" / "splits" / "cityscapes_split_seed42.json")
            print("✅ 数据划分文件已复制")

        if (DRIVE_PROJECT_DIR / "data" / "processed").exists():
            if (COLAB_WORK_DIR / "data" / "processed").exists():
                if not (COLAB_WORK_DIR / "data" / "processed").is_symlink():
                    shutil.rmtree(COLAB_WORK_DIR / "data" / "processed")
                    os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                              COLAB_WORK_DIR / "data" / "processed")
            else:
                os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                          COLAB_WORK_DIR / "data" / "processed")
            print("✅ 数据目录符号链接已创建")

        os.chdir(COLAB_WORK_DIR)

else:
    # 方案三：手动上传（解压zip文件）
    print("\n📦 使用手动上传的文件（解压模式）...")
    COLAB_WORK_DIR.mkdir(parents=True, exist_ok=True)
    os.chdir(COLAB_WORK_DIR)

    # 3. 解压项目文件（仅手动上传模式需要）
    project_zip = DRIVE_PROJECT_DIR / "project.zip"
    src_zip = DRIVE_PROJECT_DIR / "src.zip"
    data_splits_zip = DRIVE_PROJECT_DIR / "data_splits.zip"
    data_processed_zip = DRIVE_PROJECT_DIR / "data_processed.zip"

    def extract_if_needed(zip_path, target_dir, description):
        """如果目标目录不存在，则解压zip文件"""
        target_path = COLAB_WORK_DIR / target_dir
        if not target_path.exists() or not any(target_path.iterdir()):
            if zip_path.exists():
                print(f"正在解压 {description}...")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(COLAB_WORK_DIR)
                print(f"✅ {description} 解压完成！")
            else:
                print(f"⚠️  警告：未找到 {zip_path}")
                print(f"   请确保已上传文件到Google Drive，或手动解压到 {target_path}")
        else:
            print(f"✅ {description} 已存在，跳过解压")

    # 解压源代码
    if project_zip.exists():
        extract_if_needed(project_zip, ".", "项目文件")
    elif src_zip.exists():
        extract_if_needed(src_zip, "src", "源代码")
    else:
        # 尝试从已存在的目录复制
        if (DRIVE_PROJECT_DIR / "src").exists():
            print("正在复制源代码目录...")
            if (COLAB_WORK_DIR / "src").exists():
                shutil.rmtree(COLAB_WORK_DIR / "src")
            shutil.copytree(DRIVE_PROJECT_DIR / "src", COLAB_WORK_DIR / "src")
            print("✅ 源代码复制完成！")
        else:
            print("⚠️  警告：未找到源代码文件，请检查路径")

    # 解压数据划分文件
    if data_splits_zip.exists():
        extract_if_needed(data_splits_zip, "data/splits", "数据划分文件")
    elif (DRIVE_PROJECT_DIR / "data" / "splits").exists():
        print("正在复制数据划分文件...")
        (COLAB_WORK_DIR / "data" / "splits").mkdir(parents=True, exist_ok=True)
        shutil.copy2(DRIVE_PROJECT_DIR / "data" / "splits" / "cityscapes_split_seed42.json",
                     COLAB_WORK_DIR / "data" / "splits" / "cityscapes_split_seed42.json")
        print("✅ 数据划分文件复制完成！")

    # 解压图像数据（如果数据较大，建议使用符号链接而不是复制）
    if data_processed_zip.exists():
        extract_if_needed(data_processed_zip, "data/processed", "图像数据")
    elif (DRIVE_PROJECT_DIR / "data" / "processed").exists():
        print("正在创建数据目录符号链接（节省空间）...")
        (COLAB_WORK_DIR / "data").mkdir(parents=True, exist_ok=True)
        if (COLAB_WORK_DIR / "data" / "processed").exists():
            if (COLAB_WORK_DIR / "data" / "processed").is_symlink():
                print("✅ 数据目录符号链接已存在")
            else:
                shutil.rmtree(COLAB_WORK_DIR / "data" / "processed")
                os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                          COLAB_WORK_DIR / "data" / "processed")
                print("✅ 数据目录符号链接创建完成！")
        else:
            os.symlink(DRIVE_PROJECT_DIR / "data" / "processed",
                      COLAB_WORK_DIR / "data" / "processed")
            print("✅ 数据目录符号链接创建完成！")
    else:
        print("⚠️  警告：未找到图像数据，请检查路径")

# 4. 安装依赖包
print("\n📦 正在检查并安装依赖包...")

packages = [
    "torch",
    "torchvision",
    "pillow",
    "matplotlib",
    "numpy",
    "tqdm",
]

for package in packages:
    try:
        __import__(package if package != "pillow" else "PIL")
        print(f"✅ {package} 已安装")
    except ImportError:
        print(f"正在安装 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✅ {package} 安装完成")

print("\n✅ 环境设置完成！")
print(f"当前工作目录：{os.getcwd()}")
print(f"项目根目录：{COLAB_WORK_DIR}")
print(f"同步方式：{SYNC_METHOD}")

# 验证关键文件是否存在
print("\n📋 验证项目文件...")
required_files = [
    ("src/models/unet_baseline.py", "模型文件"),
    ("src/data/dataset.py", "数据集文件"),
    ("src/data/transforms.py", "数据变换文件"),
    ("data/splits/cityscapes_split_seed42.json", "数据划分文件"),
]

all_ok = True
for file_path, description in required_files:
    full_path = COLAB_WORK_DIR / file_path
    if full_path.exists():
        print(f"✅ {description}: {file_path}")
    else:
        print(f"❌ {description} 缺失: {file_path}")
        all_ok = False

if all_ok:
    print("\n🎉 所有必需文件已就绪，可以开始训练！")
else:
    print("\n⚠️  部分文件缺失，请检查同步设置")



正在挂载Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive挂载完成！

📦 从GitHub克隆项目: AltheD/Image-to-Image-Translation-Experiment
⚠️  项目目录已存在，跳过克隆

📦 正在检查并安装依赖包...
✅ torch 已安装
✅ torchvision 已安装
✅ pillow 已安装
✅ matplotlib 已安装
✅ numpy 已安装
✅ tqdm 已安装

✅ 环境设置完成！
当前工作目录：/content/Image-to-Image-Translation-Experiment
项目根目录：/content/Image-to-Image-Translation-Experiment
同步方式：github

📋 验证项目文件...
✅ 模型文件: src/models/unet_baseline.py
✅ 数据集文件: src/data/dataset.py
✅ 数据变换文件: src/data/transforms.py
✅ 数据划分文件: data/splits/cityscapes_split_seed42.json

🎉 所有必需文件已就绪，可以开始训练！


In [2]:
# 导入库
import sys
from pathlib import Path
import os

# 添加项目根目录到路径（适配Colab环境）
# 检测是否在Colab环境中运行
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 在Colab中，使用之前设置的COLAB_WORK_DIR
    try:
        ROOT = COLAB_WORK_DIR
    except NameError:
        # 如果COLAB_WORK_DIR未定义，使用默认路径
        ROOT = Path("/content/Image-to-Image-Translation-Experiment")
else:
    # 本地环境，使用相对路径
    ROOT = Path("..").resolve()

sys.path.insert(0, str(ROOT))
print(f"项目根目录：{ROOT}")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

from src.models.unet_baseline import UNetBaseline
from src.data.dataset import CityscapesDataset
from src.data.transforms import build_transform

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据路径
DATA_ROOT = ROOT / "data"
SPLIT_INDEX = ROOT / "data" / "splits" / "cityscapes_split_seed42.json"
OUTPUT_DIR = ROOT / "outputs" / "unet_baseline"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)



项目根目录：/content/Image-to-Image-Translation-Experiment
Using device: cpu


## 1. 测试U-Net模型架构



In [3]:
# 创建模型并测试前向传播
model = UNetBaseline(in_channels=3, out_channels=3).to(device)
model.eval()

# 测试输入
x = torch.randn(1, 3, 256, 256).to(device)
print(f"Input shape: {x.shape}")

with torch.no_grad():
    output = model(x)
    print(f"Output shape: {output.shape}")
    print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")
    print(f"Expected range: [-1, 1] (Tanh activation)")

# 计算参数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")



Input shape: torch.Size([1, 3, 256, 256])


AttributeError: 'UNetBlock' object has no attribute 'dropout'

## 2. 准备数据集（三种增强配置）



In [ ]:
# 定义三种增强配置
aug_configs = {
    "none": build_transform(
        image_size=256,
        jitter=False,
        horizontal_flip=False,
        color_jitter=None,
        scale_range=None,
        normalize_mode="tanh"
    ),
    "basic": build_transform(
        image_size=256,
        jitter=True,
        horizontal_flip=True,
        color_jitter=None,
        scale_range=None,
        normalize_mode="tanh"
    ),
    "strong": build_transform(
        image_size=256,
        jitter=True,
        horizontal_flip=True,
        color_jitter=(0.2, 0.2, 0.2, 0.05),
        scale_range=(0.8, 1.2),
        normalize_mode="tanh"
    ),
}

# 验证集不使用增强
val_transform = build_transform(
    image_size=256,
    jitter=False,
    horizontal_flip=False,
    color_jitter=None,
    scale_range=None,
    normalize_mode="tanh"
)

print("数据增强配置已准备完成！")



## 3. 可视化数据增强效果



In [ ]:
# 加载一个样本进行可视化
test_dataset = CityscapesDataset(
    root=DATA_ROOT,
    split="val",
    split_index=SPLIT_INDEX,
    transform=None  # 先不应用transform，看原始图像
)

# 获取原始图像
sample = test_dataset[0]
label_orig = sample["label"]
photo_orig = sample["photo"]

# 可视化原始图像和三种增强配置
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Data Augmentation Comparison", fontsize=14)

# 第一行：Label
axes[0, 0].imshow(label_orig)
axes[0, 0].set_title("Original Label")
axes[0, 0].axis("off")

for col, (aug_name, transform) in enumerate(aug_configs.items(), 1):
    # 应用增强
    label_aug, photo_aug = transform(label_orig.copy(), photo_orig.copy())

    # 转换为图像格式（tensor -> numpy）
    def tensor_to_image(tensor):
        if isinstance(tensor, torch.Tensor):
            img = tensor.permute(1, 2, 0).cpu().numpy()
        else:
            img = np.array(tensor)
        if img.min() < 0:
            img = (img + 1) / 2
        img = np.clip(img, 0, 1)
        return img

    label_img = tensor_to_image(label_aug)
    photo_img = tensor_to_image(photo_aug)

    axes[0, col].imshow(label_img)
    axes[0, col].set_title(f"Label ({aug_name})")
    axes[0, col].axis("off")

    if col == 1:
        axes[1, 0].imshow(photo_orig)
        axes[1, 0].set_title("Original Photo")
        axes[1, 0].axis("off")

    axes[1, col].imshow(photo_img)
    axes[1, col].set_title(f"Photo ({aug_name})")
    axes[1, col].axis("off")

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "augmentation_comparison.png", dpi=150, bbox_inches="tight")
plt.show()



In [ ]:
def save_triplet(label, generated, ground_truth, save_path):
    """保存三联图：Label / Generated / Ground Truth"""
    def tensor_to_image(tensor):
        if tensor.dim() == 3:
            img = tensor.permute(1, 2, 0).cpu().numpy()
        else:
            img = tensor[0].permute(1, 2, 0).cpu().numpy()
        if img.min() < 0:
            img = (img + 1) / 2
        img = np.clip(img, 0, 1)
        img = (img * 255).astype(np.uint8)
        return img

    label_img = tensor_to_image(label)
    gen_img = tensor_to_image(generated)
    gt_img = tensor_to_image(ground_truth)

    triplet = np.hstack([label_img, gen_img, gt_img])
    Image.fromarray(triplet).save(save_path)


def train_epoch(model, dataloader, criterion, optimizer, device):
    """训练一个epoch"""
    model.train()
    total_loss = 0.0
    num_batches = 0

    pbar = tqdm(dataloader, desc="Training")
    for batch in pbar:
        label = batch["label"].to(device)
        photo = batch["photo"].to(device)

        optimizer.zero_grad()
        generated = model(label)
        loss = criterion(generated, photo)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / num_batches if num_batches > 0 else 0.0


@torch.no_grad()
def validate(model, dataloader, criterion, device, num_samples=10):
    """验证并保存三联图"""
    model.eval()
    total_loss = 0.0
    num_batches = 0
    saved_samples = 0

    pbar = tqdm(dataloader, desc="Validating")
    for batch in pbar:
        label = batch["label"].to(device)
        photo = batch["photo"].to(device)
        name = batch["name"]

        generated = model(label)
        loss = criterion(generated, photo)

        total_loss += loss.item()
        num_batches += 1

        if saved_samples < num_samples:
            save_path = OUTPUT_DIR / f"sample_{saved_samples:02d}.png"
            save_triplet(label, generated, photo, save_path)
            saved_samples += 1

    return total_loss / num_batches if num_batches > 0 else 0.0



In [ ]:
# 训练配置
aug_mode = "none"
epochs = 200
batch_size = 1
lr = 2e-4
start_decay_epoch = 100

# 创建数据集
train_dataset = CityscapesDataset(
    root=DATA_ROOT,
    split="train",
    split_index=SPLIT_INDEX,
    transform=aug_configs[aug_mode]
)
val_dataset = CityscapesDataset(
    root=DATA_ROOT,
    split="val",
    split_index=SPLIT_INDEX,
    transform=val_transform
)

# 在Colab中，num_workers应该设置为0以避免多进程问题
# 在本地环境中可以使用2或更多
num_workers = 0 if IN_COLAB else 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# 创建模型
model = UNetBaseline(in_channels=3, out_channels=3).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

# 学习率调度
def get_lr_scheduler(optimizer, num_epochs, start_decay_epoch):
    def lr_lambda(epoch):
        if epoch < start_decay_epoch:
            return 1.0
        else:
            return 1.0 - (epoch - start_decay_epoch) / (num_epochs - start_decay_epoch)
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, epochs, start_decay_epoch)

# 训练循环
train_losses = []
val_losses = []

print(f"Starting training with augmentation: {aug_mode}")
for epoch in range(1, epochs + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, val_loader, criterion, device, num_samples=10)

    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {current_lr:.6f}")

        # 保存checkpoint
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_loss": train_loss,
            "val_loss": val_loss,
        }, OUTPUT_DIR / f"checkpoint_{aug_mode}_epoch_{epoch:03d}.pth")

# 保存最终模型
torch.save({
    "epoch": epochs,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "train_losses": train_losses,
    "val_losses": val_losses,
}, OUTPUT_DIR / f"model_{aug_mode}_final.pth")

print(f"\n✅ Training complete! ({aug_mode})")



### 5.3 强增强配置

> **注意**：训练强增强配置时，修改 `aug_mode = "strong"`。



## 6. 可视化训练结果

### 6.1 训练损失曲线



In [ ]:
# 加载训练历史（如果有多个配置，可以加载并对比）
# 这里展示如何绘制损失曲线

# 示例：绘制单个配置的损失曲线
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("L1 Loss")
plt.title(f"Training History ({aug_mode})")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("L1 Loss (log scale)")
plt.yscale("log")
plt.title(f"Training History - Log Scale ({aug_mode})")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f"training_curve_{aug_mode}.png", dpi=150, bbox_inches="tight")
plt.show()



### 6.2 生成结果可视化



In [ ]:
# 加载保存的三联图并展示
import glob

sample_images = sorted(glob.glob(str(OUTPUT_DIR / "sample_*.png")))[:6]

if sample_images:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Generated Samples ({aug_mode})", fontsize=14)

    for idx, img_path in enumerate(sample_images):
        row = idx // 3
        col = idx % 3
        img = Image.open(img_path)
        axes[row, col].imshow(img)
        axes[row, col].set_title(f"Sample {idx + 1}")
        axes[row, col].axis("off")

    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"samples_{aug_mode}.png", dpi=150, bbox_inches="tight")
    plt.show()
else:
    print("No sample images found. Run validation to generate samples.")



## 7. 消融实验对比（需要完成所有三种配置后）

> **提示**：完成三种增强配置的训练后，可以在这里对比结果：
> - 加载三种配置的训练历史
> - 对比损失曲线
> - 对比生成质量
> - 计算评估指标（PSNR、SSIM、MAE）

